In [39]:
import h5py
import numpy as np
from astropy.table import Table
from scipy import stats
import matplotlib.pyplot as plt

code_dir = '/Users/ctrapp/Documents/GitHub/foggie/foggie'
#data_dir = '/Users/ctrap/Documents/Foggie/'
data_dir = '/Volumes/FoggieCam/foggie_halos/'

halo_id = '008508'
run ='nref11c_nref9f'

if halo_id=="008508": GalName='Tempest'
if halo_id=="005036": GalName='Maelstrom'
if halo_id=="005016": GalName='Squall'
if halo_id=="004123": GalName='Blizzard'
if halo_id=="002392": GalName='Hurricane'
if halo_id=="002878": GalName='Cyclone'


halo_c_v_name = code_dir+"/halo_infos/"+halo_id+"/"+run+"/halo_c_v"


In [40]:
def LoadRedshiftsAndTimes(halo_c_v_name,snapshot):
    halo_c_v = Table.read(halo_c_v_name, format='ascii')

    for row in halo_c_v:
      if (snapshot[-6:] in row['col3']):
        redshift = float(row['col2'])
        time = float(row['col4'])


        return redshift,time
    
    print("Warning: could not read",halo_c_v_name)
    
    return -1,-1

In [49]:
leaf_masses = []
redshifts = []

'''
GalName='Tempest'
run='nref11c_nref9f'
snapshots=np.arange(967,2427,10)

for nsnap in snapshots:
    try:
        nsnapstring = str(nsnap).zfill(4)
        hf = h5py.File('../mass_histograms/'+GalName+"_DD"+nsnapstring+"_"+run+"_clump_mass_histogram.h5",'r')
        leaf_masses.append(hf['leaf_masses'][...])
        z,t = LoadRedshiftsAndTimes(halo_c_v_name,nsnapstring)
        redshifts.append(z)
        hf.close()
    except:
        print("Warning, could not load nsnap=",nsnap)
'''

GalNames=["Maelstrom","Squall","Blizzard","Hurricane","Cyclone","Tempest"]
run='nref11c_nref9f'
snapshots=np.arange(1060,2520,10)
for nsnap in snapshots:
    leaf_masses.append([])
    redshifts.append(-1)
    for GalName in GalNames:
        try:
            if GalName=="Tempest": nsnapstring = str(nsnap-2520+2427).zfill(4)
            else: nsnapstring = str(nsnap).zfill(4)
            hf = h5py.File('../mass_histograms/'+GalName+"_DD"+nsnapstring+"_"+run+"_clump_mass_histogram.h5",'r')
            leaf_masses[-1] = np.append(leaf_masses[-1], np.array(hf['leaf_masses']))
            z,t = LoadRedshiftsAndTimes(halo_c_v_name,nsnapstring)
            redshifts[-1] = z
            hf.close()
        except:
            print("Warning, could not load nsnap=",nsnap,'for',GalName)

Warning, could not load nsnap= 1060 for Maelstrom
Warning, could not load nsnap= 1060 for Cyclone
Warning, could not load nsnap= 1070 for Cyclone
Warning, could not load nsnap= 1080 for Maelstrom
Warning, could not load nsnap= 1080 for Blizzard
Warning, could not load nsnap= 1080 for Cyclone
Warning, could not load nsnap= 1090 for Blizzard
Warning, could not load nsnap= 1090 for Hurricane
Warning, could not load nsnap= 1090 for Cyclone
Warning, could not load nsnap= 1100 for Maelstrom
Warning, could not load nsnap= 1100 for Cyclone
Warning, could not load nsnap= 1110 for Cyclone
Warning, could not load nsnap= 1120 for Cyclone
Warning, could not load nsnap= 1120 for Tempest
Warning, could not load nsnap= 1130 for Maelstrom
Warning, could not load nsnap= 1130 for Hurricane
Warning, could not load nsnap= 1130 for Cyclone
Warning, could not load nsnap= 1140 for Blizzard
Warning, could not load nsnap= 1140 for Cyclone
Warning, could not load nsnap= 1150 for Cyclone
Warning, could not load n

In [67]:
labelsize=22
ticksize=20
titlesize=22
legendsize=16
itr=-1
for tt in range(len(leaf_masses)):
    itr+=1
    min_mass = 2
    max_mass = 10
    nbins = 100
    labelsize=18
    ticksize=18
    
    z_str = str(np.round(redshifts[tt],3))
    
    N, binedges, binnum = stats.binned_statistic(np.log10(np.array(leaf_masses[tt])), np.ones_like(np.array(leaf_masses[tt])), statistic='sum', bins=nbins, range=[min_mass,max_mass])
    dM= np.copy(N)*0
    for i in range(len(binedges)-1):
        dM[i] = 10**binedges[i+1] - 10**binedges[i]

    
    plt.figure()
    plt.plot(binedges[:-1], np.divide(N , dM), color='blue', alpha=0.7, label='Clump Mass Function', linewidth=2)
    x0 = np.array([5,6.5])
    y0 = .1*np.power(10,-2*x0+10)
    plt.plot(x0,y0,'r--',linewidth=1)
    x1 =np.array([7,8.5])
    y1 = np.power(10,-1*x1+3)
    plt.plot(x1,y1,'g--',linewidth=1)
    plt.xlabel('log Clump Mass (M$_{\\odot}$)',fontsize=labelsize)
    plt.ylabel('Clump Mass Function [M$_{\\odot}^{-1}$]',fontsize=labelsize)
    plt.legend(["ClMF","m=-2","m=-1"],fontsize=legendsize)
    #plt.xscale('log')
    plt.yscale('log')
    plt.xticks(fontsize=ticksize)
    plt.yticks(fontsize=ticksize)
    plt.xlim(2,10)
    plt.ylim(1e-9,1e0)
    plt.title("z="+z_str,fontsize=titlesize)
    plt.savefig("./AllHaloMassGifFrames/clump_mass_function_"+str(itr).zfill(4)+".png",dpi=300,bbox_inches='tight')
    plt.close()


In [68]:
from PIL import Image
import os

def create_gif_from_pngs(png_folder, output_gif, duration=500):
    """
    Compile a series of .png files into a .gif.

    Parameters:
        png_folder (str): Path to the folder containing .png files.
        output_gif (str): Path to save the output .gif file.
        duration (int): Duration for each frame in milliseconds. Default is 500ms.
    """
    # Get a sorted list of .png files in the folder
    png_files = sorted([f for f in os.listdir(png_folder) if f.endswith('.png')])
    
    if not png_files:
        raise ValueError("No .png files found in the specified folder.")
    
    # Open the images and compile them into a list
    images = [Image.open(os.path.join(png_folder, file)) for file in png_files]
    
    # Save as .gif
    images[0].save(
        output_gif,
        save_all=True,
        append_images=images[1:],
        duration=duration,
        loop=0
    )
    print(f"GIF saved at {output_gif}")


create_gif_from_pngs("./AllHaloMassGifFrames/","AllHalo_ClMF.gif",duration = 150)

GIF saved at AllHalo_ClMF.gif
